## Spectral residual algorithm

Tips:
- Saliency map can capture data fluctuation characteristic.
- The threshold algorithm can not generate good anomaly scores.

In [1]:
import sys
sys.path.append("..")

import time
import numpy as np 
import pandas as pd 

import blot
from dateutil import parser
from datetime import datetime

In [2]:
data = pd.read_csv("../data/storage.csv")
data["time"] = data.time.apply(lambda ts: datetime.fromtimestamp(ts))
# data = data.set_index(["time"])
data = data[(data["time"] > datetime(2020, 8, 30)) & (data["time"] < datetime(2020, 9, 14))]
data.head()

,time,value
3168,2020-09-11 00:00:00,1015
3169,2020-09-11 00:05:00,833
3170,2020-09-11 00:10:00,762
3171,2020-09-11 00:15:00,570
3172,2020-09-11 00:20:00,586


In [3]:
data.blot(x='time', y='value', dimensions=(1400, 900), colors={'value': 'blue'})

In [4]:
from troubleshooter.anomaly.tseries.sr import SpectralResidual
model = SpectralResidual(series=data, score_window=20, mag_window=3, threshold=2.5)
output = model.detect()

In [5]:
output.head()

,time,value,map,anomalyScore,isAnomaly
0,2020-09-11 00:00:00,1015,0.089281,0.000000,False
1,2020-09-11 00:05:00,833,0.014187,0.725766,False
2,2020-09-11 00:10:00,762,0.020769,0.498481,False
3,2020-09-11 00:15:00,570,0.115583,0.927834,False
4,2020-09-11 00:20:00,586,0.109841,0.570679,False


In [6]:
output["value"] = output["value"] / 1500.0
fig = output.blot(x='time', y=['value', 'map'], dimensions=(1400, 900), colors={'value': 'blue', 'map': 'green'}) + (output[output['isAnomaly']][['time','value']].rename(columns={"value": "anomaly"}).blot(x='time',y='anomaly', kind="scatter"))
fig.show()

In [7]:
output[output["isAnomaly"]==True]
# with open("example.html", "w") as f:
#     f.write(fig.to_html())

,time,value,map,anomalyScore,isAnomaly
37,2020-09-11 03:05:00,0.482000,0.119503,3.851932,True
63,2020-09-11 05:15:00,0.154000,0.136031,3.318373,True
265,2020-09-11 22:05:00,0.970667,0.062653,2.520644,True
268,2020-09-11 22:20:00,1.150667,0.115568,3.384163,True
288,2020-09-12 00:00:00,0.819333,0.200956,3.655803,True
351,2020-09-12 05:15:00,0.144000,0.138726,3.404960,True
543,2020-09-12 21:15:00,0.982000,0.134175,3.814211,True
556,2020-09-12 22:20:00,0.892000,0.286168,2.802359,True
576,2020-09-13 00:00:00,0.895333,0.286869,3.670131,True
613,2020-09-13 03:05:00,0.498000,0.134996,3.537782,True
